# Model evaluation

## Initialization

In [ ]:
!pip install catboost
!pip install optuna
!pip install dill
!pip install psutil
!pip install scikit-learn

In [1]:
from catboost import CatBoostClassifier, Pool
import numpy as np
import time
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import optuna
import json
import psutil
from datetime import datetime
import csv
from difflib import SequenceMatcher
from collections import defaultdict
import joblib


/Users/isofinly/Go-Goal-Track-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Batch prediction

In [2]:
categories = [
    "capacity",
    "quality",
    "view",
    "bedding",
    "balcony",
    "bedrooms",
    "club",
    "floor",
    "bathroom",
    "class",
]
models_dir = "../artifacts/models"

models = {}
label_encoders = {}
tfidf = joblib.load(f"{models_dir}/tfidf_vectorizer.joblib")

for category in categories:
    model_path = os.path.join(f"{models_dir}/cbm", f"catboost_model_{category}.cbm")
    model = CatBoostClassifier()
    model.load_model(model_path)
    models[category] = model

    le = LabelEncoder()
    le.classes_ = np.load(
        f"{models_dir}/label_text/label_encoder_{category}.npy", allow_pickle=True
    )
    label_encoders[category] = le


def batch_predict(rate_names):
    input_data = pd.Series(rate_names)
    input_tfidf = tfidf.transform(input_data)
    results = []

    for category in categories:
        predictions = models[category].predict(input_tfidf)
        predictions = predictions.ravel()
        decoded_predictions = label_encoders[category].inverse_transform(predictions)
        results.append(decoded_predictions)

    return [
        {
            category: value.item() if isinstance(value, np.integer) else value
            for category, value in zip(categories, row)
        }
        for row in zip(*results)
    ]


# Example usage
batch_examples = ["King Premium Mountain View no balcony"]

print("\nBatch prediction:")
batch_results = batch_predict(batch_examples)
for result in batch_results:
    print(json.dumps(result, indent=2))


Batch prediction:
{
  "capacity": "undefined",
  "quality": "premium",
  "view": "mountain view",
  "bedding": "undefined",
  "balcony": "balcony",
  "bedrooms": "undefined",
  "club": "not club",
  "floor": "undefined",
  "bathroom": "private bathroom",
  "class": "room"
}


## Batch prediction to a file

In [5]:
# Load the CSV file
input_file = "rates_dirty.csv"  # Replace with your input CSV file path
df_input = pd.read_csv(input_file)
df_input = df_input.fillna("undefined")


# Ensure 'rate_name' column exists
if "rate_name" not in df_input.columns:
    raise ValueError("The input CSV must contain a 'rate_name' column")

# Process in batches
batch_size = 5500
results = []

for i in range(0, len(df_input), batch_size):
    batch = df_input["rate_name"][i : i + batch_size].tolist()
    batch_results = batch_predict(batch)
    results.extend(batch_results)

# Create DataFrame from results
df_results = pd.DataFrame(results)

# Add the original rate names to the DataFrame
df_results["rate_name"] = df_input["rate_name"]

# Reorder columns to have 'rate_name' first
columns = ["rate_name"] + categories
df_results = df_results[columns]

# Save the results as a CSV file in the models_dir
output_file = os.path.join(".", "batch_prediction_results.csv")
df_results.to_csv(output_file, index=False)

print(f"\nResults have been saved to {output_file}")
print(f"Processed {len(df_input)} entries")


Results have been saved to ./batch_prediction_results.csv
Processed 180443 entries


## Find mismatches for given CSV

In [6]:
def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


def compare_csv_files(file1_path, file2_path, threshold=0.8):
    with open(file1_path, "r") as file1, open(file2_path, "r") as file2:
        reader1 = csv.DictReader(file1)
        reader2 = csv.DictReader(file2)

        total_rows = 0
        error_rows = 0
        column_errors = defaultdict(int)
        mismatches = []

        for row1, row2 in zip(reader1, reader2):
            total_rows += 1
            row_errors = 0
            row_mismatches = {}

            for cat in categories:
                val1 = row1.get(cat, "")
                val2 = row2.get(cat, "")
                val1 = "" if val1 == "undefined" else val1
                val2 = "" if val2 == "undefined" else val2

                similarity = calculate_similarity(val1, val2)
                if similarity < threshold:
                    row_errors += 1
                    column_errors[cat] += 1
                    row_mismatches[cat] = {"expected": val2, "received": val1}

            if row_errors > 0:
                error_rows += 1
                mismatches.append(
                    {
                        "row": total_rows,
                        "rate_name": row1.get("rate_name", "N/A"),
                        "mismatches": row_mismatches,
                    }
                )

        overall_error_rate = error_rows / total_rows
        print(f"\nOverall error rate: {overall_error_rate:.2f}")
        print(f"Total rows: {total_rows}")
        print(f"Rows with errors: {error_rows}")

        print("\nColumn Error Statistics:")
        for cat, error_count in column_errors.items():
            column_error_rate = error_count / total_rows
            print(
                f"Column {cat}: Error rate = {column_error_rate:.2f}, Errors = {error_count}"
            )

        with open("mismatches.json", "w") as f:
            json.dump(mismatches, f, indent=2)

        print("\nMismatches have been saved to 'mismatches.json'")


# Usage
file1_path = "rates_dirty.csv"
file2_path = "batch_prediction_results.csv"

compare_csv_files(file1_path, file2_path)


Overall error rate: 0.09
Total rows: 180443
Rows with errors: 16646

Column Error Statistics:
Column capacity: Error rate = 0.04, Errors = 7858
Column bedding: Error rate = 0.04, Errors = 7704
Column view: Error rate = 0.02, Errors = 3902
Column bathroom: Error rate = 0.01, Errors = 2509
Column class: Error rate = 0.02, Errors = 2790
Column quality: Error rate = 0.03, Errors = 4647
Column bedrooms: Error rate = 0.01, Errors = 2214
Column club: Error rate = 0.01, Errors = 1988
Column floor: Error rate = 0.01, Errors = 2012

Mismatches have been saved to 'mismatches.json'
